## Discriminate 0, 1 and 2 states

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.optimize import curve_fit
from qiskit import pulse, IBMQ, execute
from qiskit.pulse import Delay,Play
# This Pulse module helps us build sampled pulses for common pulse shapes
from qiskit.pulse import library as pulse_lib
from qiskit.tools.monitor import job_monitor


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds
qubit = 0
scale_factor = 1.e-14
mem_slot=0

In [3]:
provider = IBMQ.load_account()
backend = provider.get_backend("ibmq_armonk")

backend_name = str(backend)
print(f"Using {backend_name} backend.")
backend_defaults = backend.defaults()
backend_config = backend.configuration()
dt = backend_config.dt


Using ibmq_armonk backend.


In [4]:
def get_closest_multiple_of_16(num):
    return int(num + 8) - (int(num + 8) % 16)
# Convert seconds to dt
def get_dt_from(sec):
    return get_closest_multiple_of_16(sec/dt)

In [6]:
rough_qubit_frequency_01 = 4.97176 * GHz
pi_duration_01 = 2480
pi_amp_01 = 0.05100450494872929

rough_qubit_frequency_12 = 4.6233 * GHz
pi_duration_12 = 2480
pi_amp_12 = 0.04883121893908868

In [7]:
with pulse.build(backend=backend, default_alignment="sequential", name="discriminate_0") as zero_sched:
    # pulse.set_frequency(rough_qubit_frequency_01, pulse.drive_channel(qubit))
    pulse.measure(
        qubits=[qubit],
        registers=[pulse.MemorySlot(mem_slot)]
    )
zero_schedule = zero_sched.assign_parameters({}, inplace=False)

In [8]:
with pulse.build(backend=backend, default_alignment="sequential", name="discriminate_1") as one_sched:
    pulse.set_frequency(rough_qubit_frequency_01, pulse.drive_channel(qubit))
    pulse.play(
        pulse_lib.Constant(
            duration=pi_duration_01,
            amp=pi_amp_01,
            name="pi_pulse_01"
        ),
        pulse.drive_channel(qubit)
    )
    pulse.measure(
        qubits=[qubit],
        registers=[pulse.MemorySlot(mem_slot)]
    )
one_schedule = one_sched.assign_parameters({}, inplace=False)

In [9]:
with pulse.build(backend=backend, default_alignment="sequential", name="discriminate_2") as two_sched:
    pulse.set_frequency(rough_qubit_frequency_01, pulse.drive_channel(qubit))
    pulse.play(
        pulse_lib.Constant(
            duration=pi_duration_01,
            amp=pi_amp_01,
            name="pi_pulse_01"
        ),
        pulse.drive_channel(qubit)
    )
    pulse.set_frequency(rough_qubit_frequency_12, pulse.drive_channel(qubit))
    pulse.play(
        pulse_lib.Constant(
            duration=pi_duration_12,
            amp=pi_amp_12,
            name="pi_pulse_12"
        ),
        pulse.drive_channel(qubit)
    )
    pulse.measure(
        qubits=[qubit],
        registers=[pulse.MemorySlot(mem_slot)]
    )
two_schedule = two_sched.assign_parameters({}, inplace=False)

In [10]:
num_shots_per_exp = 2048

In [ ]:
discr_job = execute(
    [zero_schedule, one_schedule, two_schedule],
    backend=backend,
    meas_level=1,
    meas_return='single',
    shots=num_shots_per_exp
)
job_monitor(discr_job)

Job Status: job is queued (1)     

In [ ]:
job_id = discr_job.job_id()
print("JobID:", job_id)

In [ ]:
def get_results(job, num_schedules):
    res = job.result()
    measured_signals = []
    for i in range(num_schedules):
        # Get the results from the ith experiment
        results = res.get_memory(i) * 1e-14
        # Get the results for `qubit` from this experiment
        measured_signals.append(results[:, qubit])
    return np.array(measured_signals)


In [ ]:
print(get_results(discr_job, 3))

In [ ]:
IQ_012_data = get_results(discr_job, 3)
zero_data = IQ_012_data[0]
one_data = IQ_012_data[1]
two_data = IQ_012_data[2]

In [ ]:
def IQ_012_plot(x_min, x_max, y_min, y_max):
    """Helper function for plotting IQ plane for 0, 1, 2. Limits of plot given
    as arguments."""
    # zero data plotted in blue
    plt.scatter(np.real(zero_data), np.imag(zero_data), 
                    s=5, cmap='viridis', c='blue', alpha=0.5, label=r'$|0\rangle$')
    # one data plotted in red
    plt.scatter(np.real(one_data), np.imag(one_data), 
                    s=5, cmap='viridis', c='red', alpha=0.5, label=r'$|1\rangle$')
    # two data plotted in green
    plt.scatter(np.real(two_data), np.imag(two_data), 
                    s=5, cmap='viridis', c='green', alpha=0.5, label=r'$|2\rangle$')

    # Plot a large dot for the average result of the 0, 1 and 2 states.
    mean_zero = np.mean(zero_data) # takes mean of both real and imaginary parts
    mean_one = np.mean(one_data)
    mean_two = np.mean(two_data)
    plt.scatter(np.real(mean_zero), np.imag(mean_zero), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    plt.scatter(np.real(mean_one), np.imag(mean_one), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    plt.scatter(np.real(mean_two), np.imag(mean_two), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    
    plt.xlim(x_min, x_max)
    plt.ylim(y_min,y_max)
    plt.legend()
    plt.ylabel('I [a.u.]', fontsize=15)
    plt.xlabel('Q [a.u.]', fontsize=15)
    plt.title("0-1-2 discrimination", fontsize=15)

In [ ]:
x_min = -16
x_max = 0
y_min = -27.5
y_max = -7.5
IQ_012_plot(x_min, x_max, y_min, y_max)


In [ ]:
def reshape_complex_vec(vec):
    """Take in complex vector vec and return 2d array w/ real, imag entries. This is needed for the learning.
    Args:
        vec (list): complex vector of data
    Returns:
        list: vector w/ entries given by (real(vec], imag(vec))
    """
    length = len(vec)
    vec_reshaped = np.zeros((length, 2))
    for i in range(len(vec)):
        vec_reshaped[i]=[np.real(vec[i]), np.imag(vec[i])]
    return vec_reshaped


In [ ]:
# Create IQ vector (split real, imag parts)
zero_data_reshaped = reshape_complex_vec(zero_data)
one_data_reshaped = reshape_complex_vec(one_data)  
two_data_reshaped = reshape_complex_vec(two_data)  

IQ_012_data = np.concatenate((zero_data_reshaped, one_data_reshaped, two_data_reshaped))
print(IQ_012_data.shape) # verify IQ data shape


In [ ]:
# construct vector w/ 0's, 1's and 2's (for testing)
state_012 = np.zeros(num_shots_per_exp) # shots gives number of experiments
state_012 = np.concatenate((state_012, np.ones(num_shots_per_exp)))
state_012 = np.concatenate((state_012, 2*np.ones(num_shots_per_exp)))
print(len(state_012))

# Shuffle and split data into training and test sets
IQ_012_train, IQ_012_test, state_012_train, state_012_test = train_test_split(IQ_012_data, state_012, test_size=0.5)


In [ ]:
# Set up the LDA
LDA_012 = LinearDiscriminantAnalysis()
LDA_012.fit(IQ_012_train, state_012_train)


In [ ]:
# test on some simple data 
print(LDA_012.predict([[0, 0], [-10, 0], [-15, -5]]))


In [ ]:
# Compute accuracy
score_012 = LDA_012.score(IQ_012_test, state_012_test)
print(score_012)


In [ ]:
# Plot separatrix on top of scatter
def separatrixPlot(lda, x_min, x_max, y_min, y_max, shots):
    nx, ny = shots, shots

    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                         np.linspace(y_min, y_max, ny))
    Z = lda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)

    plt.contour(xx, yy, Z, [0.5], linewidths=2., colors='black')


In [ ]:
IQ_012_plot(x_min, x_max, y_min, y_max)
separatrixPlot(LDA_012, x_min, x_max, y_min, y_max, num_shots_per_exp)
